# CS 39AA - Notebook 14: Transformers for Airline Tweets w/ WandB

We'll now revisit the Airline Tweet dataset and try using one of the large pre-trained models that is available on huggingface.co. 

Note that this is roughly the same as Assign 5 Starter on Kaggle except that now we are using [Weights and Biases](https://wandb.ai) to track the training and performance of the model, and to automatically experiment with various hyperparameter values. 

In [1]:
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, TrainingArguments, Trainer
from datasets import Dataset, load_metric

In [2]:
# install wandb
#!pip install -q wandb

In [3]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: geinitz (msudenver). Use `wandb login --relogin` to force relogin


True

In [4]:
df = pd.read_csv("./data/trainA.csv")
df.head()

,sentiment,text
0,positive,@JetBlue @JayVig I like the inflight snacks! I...
1,positive,@VirginAmerica thanks guys! Sweet route over t...
2,negative,@USAirways Your exchange/credit policies are w...
3,negative,@USAirways but in the meantime I'll be sleepin...
4,negative,@VirginAmerica hold times at call center are a...


In [5]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f"device: {device}")

device: mps


In [6]:
MODEL_NAME = "bert-base-cased"
#MODEL_NAME = "bert-base-uncased"
#MODEL_NAME = "distilbert-base-uncased-finetuned-sst-2-english"
MAX_LENGTH=50

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
#model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3, max_length=MAX_LENGTH, output_attentions=False, output_hidden_states=False)

In [7]:
classes = df.sentiment.unique().tolist()
class_tok2idx = dict((v, k) for k, v in enumerate(classes))
class_idx2tok = dict((k, v) for k, v in enumerate(classes))
print(class_tok2idx)
print(class_idx2tok)

{'positive': 0, 'negative': 1, 'neutral': 2}
{0: 'positive', 1: 'negative', 2: 'neutral'}


In [8]:
def get_model():
    return AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        config=AutoConfig.from_pretrained(MODEL_NAME, num_labels=3, max_length=MAX_LENGTH, output_attentions=False, output_hidden_states=False),
    )

In [9]:
df['label'] = df['sentiment'].apply(lambda x: class_tok2idx[x])
df.head()

,sentiment,text,label
0,positive,@JetBlue @JayVig I like the inflight snacks! I...,0
1,positive,@VirginAmerica thanks guys! Sweet route over t...,0
2,negative,@USAirways Your exchange/credit policies are w...,1
3,negative,@USAirways but in the meantime I'll be sleepin...,1
4,negative,@VirginAmerica hold times at call center are a...,1


In [10]:
model = get_model()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
sequence_0 = "@united I will never fly with you again. Period."
seq0_tokens = tokenizer(sequence_0, return_tensors="pt")
print(f"number of tokens in seq0 is {len(seq0_tokens['input_ids'].flatten())}")
print(seq0_tokens)
torch.round(F.softmax(model(**seq0_tokens).logits, -1), decimals=3)

number of tokens in seq0 is 14
{'input_ids': tensor([[  101,   137, 10280,   146,  1209,  1309,  4689,  1114,  1128,  1254,
           119, 16477,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


tensor([[0.3210, 0.4050, 0.2740]], grad_fn=<RoundBackward1>)

In [12]:
sequence_1 = "@united Delayed flight, but you did the best you could. Thank you united crew."
seq1_tokens = tokenizer(sequence_1, return_tensors="pt")
print(f"number of tokens in seq1 is {len(seq1_tokens['input_ids'].flatten())}")
torch.round(F.softmax(model(**seq1_tokens).logits, -1), decimals=3)

number of tokens in seq1 is 22


tensor([[0.3140, 0.3950, 0.2910]], grad_fn=<RoundBackward1>)

In [13]:
ds_raw = Dataset.from_pandas(df[['label','text']])
ds_raw[0]

{'label': 0,
 'text': "@JetBlue @JayVig I like the inflight snacks! I'm flying with you guys on 2/28! #JVMChat"}

In [14]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)

ds = ds_raw.map(tokenize_function, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [15]:
ds[0]['text']

"@JetBlue @JayVig I like the inflight snacks! I'm flying with you guys on 2/28! #JVMChat"

In [16]:
ds = ds.shuffle(seed=42)
ds[0]['text']

'@AmericanAir 11 out of 11 delayed flights, you suck and getting worse'

In [17]:
train_prop = 0.85
ds_train = ds.select(range(int(len(ds)*train_prop)))
ds_eval = ds.select(range(int(len(ds)*train_prop), len(ds)))
print(f"len(ds_train) = {len(ds_train)}")
print(f"len(ds_eval) = {len(ds_eval)}")

len(ds_train) = 8500
len(ds_eval) = 1500


In [18]:
os.environ["WANDB_DISABLED"] = "false"
os.environ["WANDB_LOG_MODEL"] = "true"
os.environ["WANDB_PROJECT"] = "airline_tweets_w_bert"
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
os.environ["WANDB_NOTEBOOK_NAME"] = "nb14_airline_tweets_w_wandb.ipynb"


In [19]:
# define sweep config: 

# method
sweep_config = {
    'method': 'random'
}


# hyperparameters
parameters_dict = {
    'epochs': {
        'value': 3
        },
    'batch_size': {
        'values': [8, 32, 128]
        },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-5,
        'max': 1e-3
    },
    'weight_decay': {
        'values': [0.0, 0.2, 0.4, 0.6]
    },
}


sweep_config['parameters'] = parameters_dict

In [20]:
sweep_id = wandb.sweep(sweep_config, project=os.environ["WANDB_PROJECT"])

Create sweep with ID: lkfspp35
Sweep URL: https://wandb.ai/msudenver/airline_tweets_w_bert/sweeps/lkfspp35


In [21]:
#model.to(device)

In [22]:
def compute_metrics(eval_pred):
    metrics = dict()

    accuracy_metric = load_metric('accuracy')
    precision_metric = load_metric('precision')
    recall_metric = load_metric('recall')
    f1_metric = load_metric('f1')

    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    
    metrics.update(accuracy_metric.compute(predictions=preds, references=labels))
    metrics.update(precision_metric.compute(predictions=preds, references=labels, average='weighted'))
    metrics.update(recall_metric.compute(predictions=preds, references=labels, average='weighted'))
    metrics.update(f1_metric.compute(predictions=preds, references=labels, average='weighted'))
    
    return metrics


In [23]:
def train(config=None):
    with wandb.init(config=config):
        # set sweep configuration
        config = wandb.config

        # set training arguments
        training_args = TrainingArguments(
            output_dir='/Users/steve/models/airline_tweets_w_bert',
            report_to='wandb',  # Turn on Weights & Biases logging
            num_train_epochs=config.epochs,
            learning_rate=config.learning_rate,
            weight_decay=config.weight_decay,
            per_device_train_batch_size=config.batch_size,
            per_device_eval_batch_size=16,
            save_strategy='epoch', #'steps',
            evaluation_strategy='epoch', #'steps',
            eval_steps=200,
            logging_strategy='epoch',
            load_best_model_at_end=True,
            remove_unused_columns=True,
            use_mps_device=True
        )


        # define training loop
        trainer = Trainer(
            model_init=get_model,
            args=training_args,
            train_dataset=ds_train,
            eval_dataset=ds_eval,
            compute_metrics=compute_metrics
        )


        # start training loop
        trainer.train()

In [24]:
wandb.agent(sweep_id, train, count=4)

wandb: Agent Starting Run: avxkfndb with config:
wandb: 	batch_size: 128
wandb: 	epochs: 3
wandb: 	learning_rate: 5.0808901654664586e-05
wandb: 	weight_decay: 0.2


/Users/steve/opt/anaconda3/envs/py311/lib/python3.11/site-packages/transformers/training_args.py:1843: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING Config item 'learning_rate' was locked by 'sweep'

  0%|          | 0/201 [00:00<?, ?it/s]

{'loss': 0.5099, 'learning_rate': 3.387260110310972e-05, 'epoch': 1.0}


  0%|          | 0/94 [00:00<?, ?it/s]

...

  0%|          | 0/94 [00:00<?, ?it/s]

{'eval_loss': 0.3922038674354553, 'eval_accuracy': 0.8533333333333334, 'eval_precision': 0.8486748438625915, 'eval_recall': 0.8533333333333334, 'eval_f1': 0.8490752404352403, 'eval_runtime': 6.2513, 'eval_samples_per_second': 239.951, 'eval_steps_per_second': 15.037, 'epoch': 2.0}
{'loss': 0.085, 'learning_rate': 0.0, 'epoch': 3.0}


  0%|          | 0/94 [00:00<?, ?it/s]

{'eval_loss': 0.598406195640564, 'eval_accuracy': 0.862, 'eval_precision': 0.8587029949007171, 'eval_recall': 0.862, 'eval_f1': 0.8595683172403535, 'eval_runtime': 6.0685, 'eval_samples_per_second': 247.178, 'eval_steps_per_second': 15.49, 'epoch': 3.0}
{'train_runtime': 342.7683, 'train_samples_per_second': 74.394, 'train_steps_per_second': 2.328, 'train_loss': 0.2621106958030758, 'epoch': 3.0}


eval/accuracy,▅▁█
eval/f1,▆▁█
eval/loss,▁▁█
eval/precision,▇▁█
eval/recall,▅▁█
eval/runtime,█▃▁
eval/samples_per_second,▁▅█
eval/steps_per_second,▁▅█
train/epoch,▁▁▅▅███
train/global_step,▁▁▅▅███
train/learning_rate,█▅▁


In [41]:
api = wandb.Api()
sweep = api.sweep(f"msudenver/airline_tweets_w_bert/sweeps/{sweep_id}")

In [47]:
best_run = sweep.best_run(order='eval/accuracy')
best_parameters = best_run.config
print(best_parameters)
print(f"\n    batch size: {best_parameters['batch_size']}")
print(f"   learning rate: {best_parameters['learning_rate']}")
print(f"   weight decay: {best_parameters['weight_decay']}")

wandb: Sorting runs by -summary_metrics.eval/accuracy


{'bf16': False, 'fp16': False, 'fsdp': [], 'seed': 42, 'tf32': None, 'debug': [], 'optim': 'adamw_torch', 'top_k': 50, 'top_p': 1, 'epochs': 3, 'prefix': None, 'do_eval': True, 'no_cuda': False, 'use_cpu': False, 'do_train': False, 'id2label': {'0': 'LABEL_0', '1': 'LABEL_1', '2': 'LABEL_2'}, 'label2id': {'LABEL_0': 0, 'LABEL_1': 1, 'LABEL_2': 2}, 'run_name': '/Users/steve/models/airline_tweets_w_bert', 'use_ipex': False, 'adafactor': False, 'data_seed': None, 'deepspeed': None, 'do_sample': False, 'hub_token': '<HUB_TOKEN>', 'log_level': 'passive', 'max_steps': -1, 'num_beams': 1, 'ray_scope': 'last', 'report_to': ['wandb'], 'typical_p': 1, 'use_cache': True, 'adam_beta1': 0.9, 'adam_beta2': 0.999, 'batch_size': 32, 'do_predict': False, 'eval_delay': 0, 'eval_steps': 200, 'hidden_act': 'gelu', 'is_decoder': False, 'local_rank': 0, 'max_length': 50, 'min_length': 0, 'model_type': 'bert', 'optim_args': None, 'output_dir': '/Users/steve/models/airline_tweets_w_bert', 'past_index': -1, 's

Next we'll load one of the model checkpoints from the runs above and see how the results compare to those at the start of the notebook (where we added the untrained classification layer to BERT). Note that in practice we would want to take the best hyperparameter values that we saw above to train one final model (which we'd train longer, or with even more data, etc.).

In [53]:
model = AutoModelForSequenceClassification.from_pretrained('/Users/steve/models/airline_tweets_w_bert/checkpoint-67')

In [54]:
model.to('cpu')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [55]:
# recall the label encodings:
class_tok2idx

{'positive': 0, 'negative': 1, 'neutral': 2}

In [56]:
print(f"going to classify the (negative) tweet: '{sequence_0}'")
torch.round(F.softmax(model(**seq0_tokens).logits, -1), decimals=3)

going to classify the (negative) tweet: '@united I will never fly with you again. Period.'


tensor([[0.0290, 0.8410, 0.1300]], grad_fn=<RoundBackward1>)

In [57]:
print(f"going to classify the (positive) tweet: '{sequence_1}'")
torch.round(F.softmax(model(**seq1_tokens).logits, -1), decimals=3)

going to classify the (positive) tweet: '@united Delayed flight, but you did the best you could. Thank you united crew.'


tensor([[0.8320, 0.1420, 0.0260]], grad_fn=<RoundBackward1>)